<b>openworkの口コミ情報スクレイピング用プログラム</b><br>
利用にはopenworkの会員IDとパスワードが必要です。<br>
<br>取得した企業データは2つのデータフレームに格納されます。<br><br>
企業の口コミデータ⇒df_company<br>
総合的な企業の評価点や、企業情報など⇒df_company_info


In [5]:
# ※注意※
# 以下の利用規約を熟読した上で、本プログラムの利用は自己責任でお願いします。
# 利用規約　OpenWork（旧:Vorkers）
# https://www.vorkers.com/rule.php
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options # オプションを使うためには必要

# -------------------- 定数 -------------------- #
LOGIN_USER = "mk42_001@mail.goo.ne.jp"
LOGIN_PASS = "opteron252"
#企業ID.任意の企業IDを指定する (リスト形式で複数指定も可能)
company_ids = ['a0910000000FrIy']
# https://www.vorkers.com/company_answer.php?m_id=xxxxxxxxxxxxxxx

kuchikomi_ids = [1]#(リスト形式で複数指定も可能)
# https://www.vorkers.com/company_answer.php?m_id=xxxxxxxxxxxxxxx&q_no=y
#   1   組織体制・企業文化
#   2   年収・給与制度
#   3   入社理由と入社後ギャップ
#   4   働きがい・成長
#   5   女性の働きやすさ
#   6   ワーク・ライフ・バランス
#   7   -
#   8   退職検討理由
#   9   企業分析［強み・弱み・展望］
#   10  経営者への提言

now = datetime.datetime.now()
# OUTPUT_CSV_FILEPATH = now.strftime("./%Y%m%d-%H%M.csv")
#OUTPUT_CSV_FILEPATH = "./scraping.csv"

DRIVER_PATH = './chromedriver'
BASE_URL = "https://www.vorkers.com/"

#wait time. いじらない
SLEEP_TIME = 5

#指定したページ数(1ページ25コメント)だけコメントを取得する。
#上限なしの場合は10000とかテキトーに大きい数を入れる
MAX_PAGES = 1

#Error Message. 要素を取得できない場合にNullで埋める
Error_Message = None
# --------------------------------------------- #
 
#option = Options()   # オプションを用意
#option.add_argument('--headless')   #headless mode にはしない 
driver = webdriver.Chrome(executable_path = DRIVER_PATH)#options=option)
driver.get(BASE_URL) #Top pageにアクセス
time.sleep(SLEEP_TIME) #sleep timeを指定

# login page
driver.find_element_by_xpath('//*[@id="headerMenu"]/li[3]/a').click()
time.sleep(SLEEP_TIME)

# login
user_box = driver.find_element_by_xpath('//*[@id="_username"]')
pass_box = driver.find_element_by_xpath('//*[@id="_password"]')
user_box.send_keys(LOGIN_USER)
pass_box.send_keys(LOGIN_PASS)
driver.find_element_by_xpath('//*[@id="log_in"]').click()
time.sleep(SLEEP_TIME)

###スクレイピング項目の中身の入るハコを作る##
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
g=[]
h=[]

#こっちは企業情報が入るハコ
#k0=[]
k1=[]
k2=[]
k3=[]
k4=[]
k5=[]
k6=[]
k7=[]
k8=[]
k9=[]
k10=[]
k11=[]
k12=[]
k13=[]
#####################

# scraping1 口コミ＋評点
for company_id in company_ids:
    for kuchikomi_id in kuchikomi_ids:
        kuchikomi_start_url = BASE_URL + "company_answer.php?m_id=" + company_id + "&q_no=" + str(kuchikomi_id)

        page = 1
        
        while True:
            
            kuchikomi_now_url = kuchikomi_start_url + "&next_page=" + str(page)
            driver.get(kuchikomi_now_url)
            time.sleep(SLEEP_TIME)

            ######### エラー処理 #########
            # 最終ページ＋1ページ目は1ページ目に戻ってくる
            if driver.current_url == kuchikomi_start_url and page != 1:
                break

            # 1ページしかない場合は、2ページ目はエラー
            elif driver.title == "404エラー　File not found　OpenWork":
                break
                
            #指定したページ数に達したら終了
            elif page > MAX_PAGES:
                break
            ###########################
            

            ##### スクレイピング処理 #####
            print("[---------- " + driver.title + " ----------]")
            # articleは1ページ最大25件
            for i in range(1, 26):
                
                print("[ article " + str(i) + " ]")

                # 1. 回答日（2018年05月17日）
                answer_time_xpath  = '//*[@id="anchor01"]/article[' + str(i) + ']/div[1]/p/time'
                # 2. 回答者属性（SE、在籍3年未満、現職（回答時）、新卒入社、男性、xxx）
                answer_type_xpath  = '//*[@id="anchor01"]/article[' + str(i) + ']/div[2]/dl/dt/a'
                # 3. 点数（2.4）
                answer_score_xpath = '//*[@id="anchor01"]/article[' + str(i) + ']/div[2]/dl/dt/span[2]/span[6]'
                # 4. 本文
                answer_body_xpath  = '//*[@id="anchor01"]/article[' + str(i) + ']/div[2]/dl/dd'
                # 5 業種
                company_type_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[1]/ul/li/a'
                # 6 企業名
                company_name_xpath = '//*[@id="anchor01"]/div[1]/h3'

                try:
                    answer_time_text  = driver.find_element_by_xpath(answer_time_xpath).text
                    answer_type_text  = driver.find_element_by_xpath(answer_type_xpath).text
                    answer_score_text = driver.find_element_by_xpath(answer_score_xpath).text
                    answer_body_text  = driver.find_element_by_xpath(answer_body_xpath).text
                    company_type_text = driver.find_element_by_xpath(company_type_xpath).text
                    company_name_text = driver.find_element_by_xpath(company_name_xpath).text
                # articleが25件未満
                except Exception:
                # except selenium.common.exceptions.NoSuchElementException:
                    break
                
                #空リストに値を挿入
                a+=[company_name_text]
                b+=[company_id]
                c+=[str(kuchikomi_id)]
                d+=[answer_time_text]
                e+=[answer_type_text]
                f+=[answer_score_text]
                g+=[answer_body_text]
                h+=[company_type_text]
                
                #with open(OUTPUT_CSV_FILEPATH, mode='a',encoding='utf_8_sig') as f:
                    #f.write(tmp_str)
            ###########################

            page += 1
            
    # scraping2 (企業基本データ+会社評価スコアTop)
    for company_id in company_ids:
        kuchikomi_top_url = BASE_URL + "company.php?m_id=" + company_id
        driver.get(kuchikomi_top_url)
        time.sleep(SLEEP_TIME)
        print('--TOP page--')
        # 1. 残業時間
        overtime_xpath  = '//*[@id="mainColumn"]/article[1]/div[2]/div[1]/div[2]/dl[1]/dd/span'
        #2. 有給消化率
        paidvacation_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[1]/div[2]/dl[2]/dd/span'
        #3. 平均年収
        average_salary_xpath = '//*[@id="mainColumn"]/article[3]/div[2]/table/tbody/tr[1]/td/span[1]'
        #4. 設立年
        #エキスパンドのxpath
        expand_xpath = '//*[@id="jsCompanyInfoBox"]/p/a'
        #「もっとみる」をクリックしてエキスパンド
        driver.execute_script('arguments[0].click();', driver.find_element_by_xpath(expand_xpath))
        #Wait
        time.sleep(SLEEP_TIME)
        
        #設立年のxpathが異なるケースがあるため、分岐させる
        if '年' in driver.find_element_by_xpath('//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[5]').text:
            established_year_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[5]'
        elif '年' in driver.find_element_by_xpath('//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[4]').text:
            established_year_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[4]'
        #そもそも年がそんざいしないため、取得できない場合は、業種をダミーデータで入れとく
        else:
            established_year_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[1]'
            
        #5.総合スコア
        totalscore_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[1]/div[1]/p[2]/span'
        
        #6.社員の士気
        motivation_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[2]/dl/dd'
        
        #7.風通し良さ
        openness_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[3]/dl/dd'

        #8.社員相互尊重
        respect_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[4]/dl/dd'
        
        #9.20代成長性
        twenties_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[5]/dl/dd'
        
        #10.人材長期育成
        longterm_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[6]/dl/dd'
        
        #11.法令遵守
        compliance_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[7]/dl/dd'
        
        #12.人事評価適正感
        fairness_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[7]/dl/dd'
        
        #13. 待遇満足度
        salary_xpath = '/html/body/div[1]/main/div/div[1]/article[1]/div[2]/div[2]/ul/li[1]/dl/dd'    
        
        #テキスト取得(エレメントが存在しない場合があるため、全要素において例外処理をいれる)
        try:
            overtime_text = driver.find_element_by_xpath(overtime_xpath).text
        except Exception:
            overtime_text = Error_Message
        
        try:
            paidvacation_text  = driver.find_element_by_xpath(paidvacation_xpath).text
        except Exception:
            paidvacation_text = Error_Message
            
        try:
            average_salary_text = driver.find_element_by_xpath(average_salary_xpath).text
        except Exception:
            average_salary_text = Error_Message
            
        #設立年はトライ処理必要ない
        established_year_text = driver.find_element_by_xpath(established_year_xpath).text

        try:
            totalscore_text = driver.find_element_by_xpath(totalscore_xpath).text
        except Exception:
            totalscore_text = Error_Message
            
        try:
            motivation_text = driver.find_element_by_xpath(motivation_xpath).text
        except Exception:
            motivation_text = Error_Message
            
        try:
            openness_text = driver.find_element_by_xpath(openness_xpath).text
        except Exception:
            openness_text = Error_Message
            
        try:
            respect_text = driver.find_element_by_xpath(respect_xpath).text
        except Exception:
            respect_text = Error_Message
            
        try:
            twenties_text = driver.find_element_by_xpath(twenties_xpath).text
        except Exception:
            twenties_text = Error_Message
            
        try:
            longterm_text = driver.find_element_by_xpath(longterm_xpath).text
        except Exception:
            longterm_text = Error_Message
        
        try:
            compliance_text = driver.find_element_by_xpath(compliance_xpath).text
        except Exception:
            compliance_text = Error_Message
        
        try:
            fairness_text = driver.find_element_by_xpath(fairness_xpath).text
        except Exception:
            fairness_text = Error_Message
        
        try:
            salary_text = driver.find_element_by_xpath(salary_xpath).text
        except Exception:
            salary_text = Error_Message

        #空リストに値代入
        #k0.append(company_name_text)
        k1.append(overtime_text)
        k2.append(paidvacation_text)
        k3.append(average_salary_text)
        #established_yearが設立年を正しく取れていないケースがあるので、取れていない場合はk4にNoneを追加
        #正しく取れているのであれば、「年」を除去した上で、int型に直してからk4に追加
        if '年' in established_year_text:
            established_year_text = int(established_year_text.replace('年',''))
            k4.append(established_year_text)
        else:
            k4.append(None)
        k5.append(totalscore_text)
        k6.append(motivation_text)
        k7.append(openness_text)
        k8.append(respect_text)
        k9.append(twenties_text)
        k10.append(longterm_text)
        k11.append(compliance_text)
        k12.append(fairness_text)
        k13.append(salary_text)

driver.quit()
df_company = pd.DataFrame(data={'企業名':a, '企業ID': b, '口コミ種別ID': c,'回答日時': d, '回答タイプ': e,'評価':f,'本文':g,'業種':h},columns=['企業名', '企業ID', '口コミ種別ID', '回答日時', '回答タイプ','評価','本文','業種'])



#df_company_infoのハコをまずつくる
df_company_info = pd.DataFrame(index=[],columns=['企業名','月残業時間','有給消化率','平均年収','設立年','総合スコア',
                                    '社員の士気','風通し','社員相互尊重',
                                    '20代成長性','長期育成','コンプラ',
                                    '評価納得感','待遇満足度'])

#company_idsのリストの要素長さ分だけ、dataframeへappendを行う
for i in range(len(company_ids)):
    df_company_info = df_company_info.append({'企業名': np.unique(a)[i], '月残業時間': k1[i],
                            '有給消化率':k2[i],
                            '平均年収':k3[i],'設立年':k4[i],'総合スコア':k5[i],
                            '社員の士気':k6[i],'風通し':k7[i],'社員相互尊重':k8[i],
                            '20代成長性':k9[i],'長期育成':k10[i],'コンプラ':k11[i],
                            '評価納得感':k12[i],'待遇満足度':k13[i]
                            },ignore_index=True)

#df_companyに回答種別をくっつける###
answer_type={'1':"組織体制・企業文化",'2':"年収・給与制度",'3':"入社理由と入社後ギャップ",'4':"働きがい・成長",
             '5':"女性の働きやすさ",'6':"ワーク・ライフ・バランス",'8':"退職検討理由",'9':"企業分析［強み・弱み・展望］",
            '10':"経営者への提言"}

df_company["口コミ種別"] = df_company.口コミ種別ID.map(answer_type)
##############################

####回答タイプから、職種、在籍年数、現職かどうか、新卒かどうか、性別を抜き出す
job_type = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,0]
years = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,1]
stay = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,2]
new_graduate = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,3]
sex = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,4]

#df_companyに新しい列として追加
df_company['職種'] = job_type
df_company['経験年数'] = years
df_company['現職/退職'] = stay
df_company['新卒/中途'] = new_graduate
df_company['性別'] = sex

#元の回答タイプのカラムはいらないので落とす
df_company.drop("回答タイプ",axis=1,inplace=True)

#本文の改行コード削除
df_company.本文 = df_company.本文.replace('\n','',regex=True)

####結果のshape
print(df_company.shape)
print(df_company_info.shape)

[---------- 日立建機の「組織体制・企業文化」 OpenWork（旧:Vorkers） ----------]
[ article 1 ]
[ article 2 ]
[ article 3 ]
[ article 4 ]
[ article 5 ]
[ article 6 ]
[ article 7 ]
[ article 8 ]
[ article 9 ]
[ article 10 ]
[ article 11 ]
[ article 12 ]
[ article 13 ]
[ article 14 ]
[ article 15 ]
[ article 16 ]
[ article 17 ]
[ article 18 ]
[ article 19 ]
[ article 20 ]
[ article 21 ]
[ article 22 ]
[ article 23 ]
[ article 24 ]
[ article 25 ]
--TOP page--
(25, 13)
(1, 14)


In [6]:
#年収をint、月残業時間をfloatに、有給消化率はfloatに直したいが、取得できなかった場合はNoneが入っているので
#該当するカラムがNoneではない行に対してのみ、置換を行う
df_company_info[df_company_info['平均年収'].notna()].平均年収 = df_company_info[df_company_info['平均年収'].notna()].平均年収.astype('int')
df_company_info[df_company_info['月残業時間'].notna()].月残業時間 = df_company_info[df_company_info['月残業時間'].notna()].月残業時間.astype('float')
df_company_info[df_company_info['有給消化率'].notna()].有給消化率 = df_company_info[df_company_info['有給消化率'].notna()].有給消化率.astype('float')

#正しく取得できているかを確認
display(df_company.head())
display(df_company_info.head())

,企業名,企業ID,口コミ種別ID,回答日時,評価,本文,業種,口コミ種別,職種,経験年数,現職/退職,新卒/中途,性別
0,日立建機株式会社,a0910000000FrIy,1,2021年09月05日,3.0,基本的にはニッポンの大手企業のイメージそのもの。上の方々の考えを確認しつつ、その意見に沿うよ...,機械関連,組織体制・企業文化,開発設計,在籍3～5年,現職（回答時）,中途入社,男性
1,日立建機株式会社,a0910000000FrIy,1,2021年08月25日,3.5,日立グループ会社があり、社内文化雰囲気などは本体と大きく変わりません。建機分野は狭くて、建設...,機械関連,組織体制・企業文化,設計,在籍5～10年,退社済み(2015年より前),中途入社,男性
2,日立建機株式会社,a0910000000FrIy,1,2021年05月11日,2.6,日立建機は従業員に対して健康、安全、コンプライアンスそれぞれに高い意識を持つよう働きかけてお...,機械関連,組織体制・企業文化,開発,技術,係長以上,在籍20年以上,現職（回答時）
3,日立建機株式会社,a0910000000FrIy,1,2020年06月16日,2.8,・旧態依然・朝令暮改という一見矛盾に思える企業体質はもう10年も変わっていないように思える。...,機械関連,組織体制・企業文化,管理部門,事務スタッフ,一般,在籍10～15年,現職（回答時）
4,日立建機株式会社,a0910000000FrIy,1,2022年02月05日,3.9,とにかく古い会社。コロナ禍の緊急事態宣言中などはさすがにテレワークをしていたが、それが終了し...,機械関連,組織体制・企業文化,コーポレート,在籍10～15年,退社済み(2020年以降),新卒入社,男性


,企業名,月残業時間,有給消化率,平均年収,設立年,総合スコア,社員の士気,風通し,社員相互尊重,20代成長性,長期育成,コンプラ,評価納得感,待遇満足度
0,日立建機株式会社,26.3,53.5,584,1970,3.17,2.7,3.1,3.0,2.8,2.7,4.3,4.3,3.5


In [7]:
#SqLiteに書き込む
import sys
import sqlite3

sys.stderr.write("*** 開始 ***\n")

#書き込み先db指定
file_company = "./datas/OpenWork_company.db"
file_company_info = './datas/OpenWork_company_info.db'

#接続
conn = sqlite3.connect(file_company)
conn2 = sqlite3.connect(file_company_info)
#
#df_companyをsqlに追加
df_company.to_sql('df_company',conn,if_exists='append',index=None)
df_company_info.to_sql('df_company_info',conn2,if_exists='append',index=None)
#
conn.close()
conn2.close()
#
sys.stderr.write("*** 終了 ***\n")

*** 開始 ***
*** 終了 ***


11

In [8]:
#SQLiteからの読み込み
import sys
import pandas as pd
#
import sqlite3
# ------------------------------------------------------------------
sys.stderr.write("*** 開始 ***\n")
#読み込み先
file_company = "./datas/OpenWork_company.db"
file_company_info = './datas/OpenWork_company_info.db'

conn = sqlite3.connect(file_company)
conn2 = sqlite3.connect(file_company_info)
#
df_company=pd.read_sql_query('SELECT * FROM df_company', conn)
df_company_info=pd.read_sql_query('SELECT * FROM df_company_info', conn2)
display(df_company.head(15))
display(df_company_info.head(15))
#
conn.close()
conn2.close()
sys.stderr.write("*** 終了 ***\n")

*** 開始 ***


,企業名,企業ID,口コミ種別ID,回答日時,評価,本文,業種,口コミ種別,職種,経験年数,現職/退職,新卒/中途,性別
0,日立建機株式会社,a0910000000FrIy,1,2021年09月05日,3.0,基本的にはニッポンの大手企業のイメージそのもの。上の方々の考えを確認しつつ、その意見に沿うよ...,機械関連,組織体制・企業文化,開発設計,在籍3～5年,現職（回答時）,中途入社,男性
1,日立建機株式会社,a0910000000FrIy,1,2021年08月25日,3.5,日立グループ会社があり、社内文化雰囲気などは本体と大きく変わりません。建機分野は狭くて、建設...,機械関連,組織体制・企業文化,設計,在籍5～10年,退社済み(2015年より前),中途入社,男性
2,日立建機株式会社,a0910000000FrIy,1,2021年05月11日,2.6,日立建機は従業員に対して健康、安全、コンプライアンスそれぞれに高い意識を持つよう働きかけてお...,機械関連,組織体制・企業文化,開発,技術,係長以上,在籍20年以上,現職（回答時）
3,日立建機株式会社,a0910000000FrIy,1,2020年06月16日,2.8,・旧態依然・朝令暮改という一見矛盾に思える企業体質はもう10年も変わっていないように思える。...,機械関連,組織体制・企業文化,管理部門,事務スタッフ,一般,在籍10～15年,現職（回答時）
4,日立建機株式会社,a0910000000FrIy,1,2022年02月05日,3.9,とにかく古い会社。コロナ禍の緊急事態宣言中などはさすがにテレワークをしていたが、それが終了し...,機械関連,組織体制・企業文化,コーポレート,在籍10～15年,退社済み(2020年以降),新卒入社,男性
5,日立建機株式会社,a0910000000FrIy,1,2022年02月05日,3.1,入社後は学歴など関係なく、能力のある社員にはしっかりとチャンスが与えられる。主任クラスでも役...,機械関連,組織体制・企業文化,営業本部,営業,在籍5～10年,退社済み(2020年より前),中途入社
6,日立建機株式会社,a0910000000FrIy,1,2022年02月02日,3.0,所属している部署は正社員と派遣社員の割合が半々くらいで、派遣社員が会社の繁忙期に比例して増減...,機械関連,組織体制・企業文化,製造部,製造,一般作業員,在籍3～5年,現職（回答時）
7,日立建機株式会社,a0910000000FrIy,1,2021年06月05日,4.0,安定していると感じます。大きな会社であるため組織体制は整っています。組織の変更も頻繁にあり、...,機械関連,組織体制・企業文化,IT,在籍10～15年,退社済み(2020年より前),新卒入社,男性
8,日立建機株式会社,a0910000000FrIy,1,2021年03月10日,2.5,組織体制：組織としては基本的に縦割りになっている。部門間のコラボレーションなどはあまりない印...,機械関連,組織体制・企業文化,事務系,在籍5～10年,現職（回答時）,中途入社,男性
9,日立建機株式会社,a0910000000FrIy,1,2020年01月11日,3.4,過去は日立グループの一社であるにも関わらず独立性が高い企業であったが、ここ２〜３年の事業環境...,機械関連,組織体制・企業文化,管理部門,在籍3～5年,現職（回答時）,新卒入社,男性


,企業名,月残業時間,有給消化率,平均年収,設立年,総合スコア,社員の士気,風通し,社員相互尊重,20代成長性,長期育成,コンプラ,評価納得感,待遇満足度
0,日立建機株式会社,26.3,53.5,584,1970,3.17,2.7,3.1,3.0,2.8,2.7,4.3,4.3,3.5


*** 終了 ***


11